<a href="https://colab.research.google.com/github/harshavardhanyadav2004/Face-Attendance-Server/blob/main/Its_running_dhg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=89ec6cf293227eb62415d8fc861cf86f082a46a17df8a8ae954b96d2a8b051b3
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [2]:
pip install pyngrok

In [3]:
pip install flask-bootstrap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.4/456.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460120 sha256=e97e39e1353f0f33837e2d77a48fcfec17691694404963ae806e17b976b89c39
  Stored in directory: /root/.cache/pip/wheels/6f/33/ad/26540e84a28334e5dfeda756df270f95353779f03bc5cf40d4
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3928 sha256=65a55df6ce9763eab585df9191d9a894851599f4938448a671cc575170fc74c6
  Stored in directory: /root/.cache/pip/wheels/19/31/99/2ec5b4459cac4d801d6201d501a354366d180afc9f8bb2d333
Successfully built flask-bootstrap visitor


In [4]:
port_no = 5000

In [5]:
import face_recognition as fr
import cv2
import os
import pickle
import numpy as np

class FaceRecognition:
    def __init__(self, frames_dir, known_face_encodings, student_dic):
        self.frames_dir = frames_dir
        self.known_face_encodings = known_face_encodings
        self.students = student_dic

        csm_students = list(self.students.keys())

    def load_image_paths(self):
        image_paths = [os.path.join(self.frames_dir, filename) for filename in os.listdir(self.frames_dir) if filename.lower().endswith('.jpg')]
        return image_paths

    def load_new_student(path):
        x_image = fr.load_image_file(path)
        print(x_image)
        x_face_encoding = fr.face_encodings(x_image)[0]
        return x_face_encoding

    def find_faces(self, image_paths):
        face_locations = []
        face_encodings = []

        for path in image_paths:
            frame = fr.load_image_file(path)
            rgb_frame = frame[:, :, ::-1]  # Convert BGR to RGB

            face_locations = fr.face_locations(rgb_frame, number_of_times_to_upsample=1, model='hog')
            face_encodings_for_frame = fr.face_encodings(frame, face_locations)

            face_encodings.extend(face_encodings_for_frame)

        return face_encodings



    def cosine_similarity(self, vector_a, vector_b):
        vector_a = np.array(vector_a, dtype=float)
        vector_b = np.array(vector_b, dtype=float)
        dot_product = np.dot(vector_a, vector_b)
        magnitude_a = np.linalg.norm(vector_a)
        magnitude_b = np.linalg.norm(vector_b)

        if magnitude_a == 0 or magnitude_b == 0:
            return 0

        cosine_similarity_value = dot_product / (magnitude_a * magnitude_b)
        return cosine_similarity_value

    def compare_faces(self, video_face_encodings):
        similarities = []
        student_names = []
        csm_students = list(self.students.keys())

        for y in video_face_encodings:
            nec = [self.cosine_similarity(y, x) for x in self.known_face_encodings]
            similarities.append(nec)

        for x in similarities:
            student_names.append(csm_students[np.argmax(x)])

        return list(set(student_names))



In [6]:
import face_recognition as fr
import os
import pickle
import numpy as np
import cv2 as cv

class VideoPreprocessor:
    def __init__(self, directory, video_path):
        self.directory = directory
        self.video_path = video_path
        self.fps = None
        self.frame_count = 0
        self.extracted_frame_count = 0

    def empty_folder(self):
        for file in os.listdir(self.directory):
            file_path = os.path.join(self.directory, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        return True

    def extract_frames_per_second(self):
        try:
            cap = cv.VideoCapture(self.video_path)
            if not cap.isOpened():
                raise IOError("Error opening video!")

            self.fps = cap.get(cv.CAP_PROP_FPS)
            if self.fps <= 0:
                raise ValueError("Invalid FPS value.")

            if self.empty_folder():
                while True:
                    ret, frame = cap.read()

                    if not ret:
                        print("End of video reached.")
                        break

                    # Extract one frame per second
                    if self.frame_count % int(self.fps) == 0:
                        filename = f"{self.directory}/frame_{self.extracted_frame_count}.jpg"
                        cv.imwrite(filename, frame)
                        self.extracted_frame_count += 1

                    self.frame_count += 1

                print(f"Extracted {self.extracted_frame_count} frames to {self.directory}")

        except IOError as e:
            print(f"IOError: {e}")
        except ValueError as e:
            print(f"ValueError: {e}")
        finally:
            cap.release()

# Example usage
'''if __name__ == "__main__":
    directory = "Frames"
    video_path = "video.mp4"

    if not os.path.exists(directory):
        os.makedirs(directory)

    preprocessor = VideoPreprocessor(directory, video_path)
    preprocessor.extract_frames_per_second()'''



'if __name__ == "__main__":\n    directory = "Frames"\n    video_path = "video.mp4"\n\n    if not os.path.exists(directory):\n        os.makedirs(directory)\n\n    preprocessor = VideoPreprocessor(directory, video_path)\n    preprocessor.extract_frames_per_second()'

In [ ]:
from flask import *
import os
from pyngrok import ngrok
ngrok.set_auth_token("2fhOSoa9jSR3KoJHNOmgUultPE2_6jkfdizMSEioQtYY1Q35s")
public_url = ngrok.connect(5000).public_url
import sqlite3
import numpy as np
new_List = []
app = Flask(__name__)
app.secret_key='I_AM_IRON_MAN'
ALLOWED_EXTENSIONS = ['mp4']
@app.route("/")
def index():
    return render_template("newForm.html")
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
@app.route("/uploadStudents",methods=["POST","GET"])
def ImageUpload():
    if request.method == 'POST':
        name = request.form["Name"]
        roll = request.form["Roll"]
        image = request.files['Image']
        image.save(image.filename)
        print(image)
        face_encoded = FaceRecognition.load_new_student(image)
        new_List.append(face_encoded)
        new_array = "/".join([str(i) for i in face_encoded])
        if SelectFromTable(roll):
            return jsonify("Already present in the database")
        with sqlite3.connect("Attendance.db") as conn :
            cursor = conn.cursor()
            cursor.execute("INSERT INTO USERS (Name,Roll,Face_Encodings) VALUES (?,?,?)",(name,roll,new_array))
            conn.commit()
        return jsonify("SuccessFully Uploaded")
@app.route("/uploadVideo")
def videoUpload():
  return render_template("index.html")
@app.route("/uploadDetailedVideo",methods = ["POST","GET"])
def detailedVideoUpload():
    if 'video' not in request.files:
        return  jsonify("No Video Found")
    video = request.files['video']
    if video.filename == "":
        return jsonify("No File Selected")
    if video and allowed_file(video.filename):
        video.save(video.filename)
        video_object = VideoPreprocessor("/content/Frames",video_path=video.filename)
        if video_object.empty_folder():
            video_object.extract_frames_per_second()
            studentdetails,known_encodings = fetchFromDataBase()
            face_object = FaceRecognition("/content/Frames",known_encodings,studentdetails)
            new_image_paths = face_object.load_image_paths()
            new_face_encodings = face_object.find_faces(new_image_paths)
            attendees_list = face_object.compare_faces(new_face_encodings)
            new_dictionary = joining_sequel_function(attendees_list)
            return jsonify(new_dictionary)
        return jsonify("Error in emptying the Folder")
    return jsonify("Invalid type in the File ")
def joining_sequel_function(attendees_list):
  with sqlite3.connect("Attendance.db") as conn:
    print(attendees_list)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM USERS")
    all_attendees_list = { i[1] : i[0] for i in cursor }
    print(all_attendees_list)
    abseentees_list = [ i for i in all_attendees_list.keys() if i not in attendees_list ]
    print(abseentees_list)
    dictionary = {}
    array_presents = []
    array_absents  = []
    for i in attendees_list:
      array_presents.append({"name":all_attendees_list[i],"roll":i})
    dictionary["presentees"] = array_presents
    for i in abseentees_list:
      array_absents.append({"name":all_attendees_list[i],"roll":i})
    dictionary["absentees"] = array_absents
    print(dictionary)
    return dictionary
def fetchFromDataBase():
    with sqlite3.connect("Attendance.db") as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM USERS")
        student_dict = {}
        new_Face_Encodings = []
        for i in cursor :
            student_dict[i[1]] = i[0]
            new_Face_Encodings.append(np.array([float(j) for j in i[2].split("/") ]))
    return student_dict,new_Face_Encodings
@app.route("/generatePDF",methods = ["POST","GET"])
def lastTouch():
  if request.method == "POST":
      get_text = request.form["textdata"]
      new_text_speech = generator(get_text)
      return jsonify(new_text_speech)
def createTable():
    with sqlite3.connect("Attendance.db") as conn:
        cursor = conn.cursor()
        cursor.execute("CREATE TABLE USERS (Name VARCHAR(255), Roll VARCHAR(255), Face_Encodings TEXT(16000) , PRIMARY KEY(Roll))")
    print("created successfully")
def SelectFromTable(roll_no):
    with sqlite3.connect("Attendance.db") as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM USERS ORDER BY Roll")
        another_tuple = []
        for i in cursor:
            another_tuple.append(i[1])
        if roll_no in another_tuple:
            return True
           # https://8950-35-185-178-154.ngrok-free.app
    return False
if __name__ == '__main__':
    #createTable()
    print(public_url)
    app.run(port = port_no )

https://af74-34-68-47-86.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


End of video reached.
Extracted 3 frames to /content/Frames


INFO:werkzeug:127.0.0.1 - - [01/May/2024 16:46:16] "POST /uploadDetailedVideo HTTP/1.1" 200 -


['21981a4465']
{'21981a4465': 'Sudha Sankar rao', '21981a4466': 'Harsha Vardhan'}
['21981a4466']
{'presentees': [{'name': 'Sudha Sankar rao', 'roll': '21981a4465'}], 'absentees': [{'name': 'Harsha Vardhan', 'roll': '21981a4466'}]}
End of video reached.
Extracted 3 frames to /content/Frames


INFO:werkzeug:127.0.0.1 - - [01/May/2024 16:50:08] "POST /uploadDetailedVideo HTTP/1.1" 200 -


['21981a4466']
{'21981a4465': 'Sudha Sankar rao', '21981a4466': 'Harsha Vardhan'}
['21981a4465']
{'presentees': [{'name': 'Harsha Vardhan', 'roll': '21981a4466'}], 'absentees': [{'name': 'Sudha Sankar rao', 'roll': '21981a4465'}]}


In [21]:
attendees_list=["21981a4465"]
with sqlite3.connect("Attendance.db") as conn:
    with sqlite3.connect("Attendance.db") as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM USERS")
        student_dict = {}
        new_Face_Encodings = []
        for i in cursor :
            student_dict[i[1]] = i[0]
            new_Face_Encodings.append(np.array([float(j) for j in i[2].split("/") ]))
        print(student_dict,new_Face_Encodings)






{'21981a4465': 'Sudha Sankar rao', '21981a4466': 'Harsha Vardhan'} [array([-0.13409157,  0.13800862,  0.05355558, -0.05370569, -0.06824048,
       -0.04218274, -0.03351473, -0.10006957,  0.17636909, -0.09130377,
        0.23674241, -0.08588597, -0.20948203, -0.12220045,  0.02624761,
        0.09816415, -0.13268098, -0.1231949 , -0.02000012, -0.14298905,
        0.02302869,  0.01699419,  0.02203441,  0.02601515, -0.19595644,
       -0.3748391 , -0.09661537, -0.13424091, -0.02401798, -0.05964912,
       -0.01487449,  0.06595365, -0.1990841 , -0.01786499,  0.00695241,
        0.15933605, -0.02809027,  0.0147193 ,  0.15072964, -0.00298209,
       -0.15610576, -0.05875939,  0.05332009,  0.22004597,  0.1062866 ,
        0.09089354,  0.03621432, -0.01376179,  0.07284073, -0.11803743,
        0.10290713,  0.17175156,  0.09627068,  0.03114554,  0.07370009,
       -0.13480112, -0.00356767,  0.02902903, -0.1969893 , -0.02877663,
        0.07093938, -0.06295201, -0.11513643,  0.0131266 ,  0.255254

In [ ]:
pip -q install crewai langchain_groq langchain_community duckduckgo-search

In [ ]:
from langchain_groq import ChatGroq
Groq_llm = ChatGroq(api_key = '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', model = 'gemma-7b-it')
from crewai import Crew, Agent, Task, Process
from langchain_community.tools import DuckDuckGoSearchRun

In [ ]:
import pprint

In [ ]:
from langchain.agents import load_tools

search_tool = DuckDuckGoSearchRun()

In [ ]:
import json  # Import the JSON module to parse JSON strings
from langchain_core.agents import AgentFinish

agent_finishes  = []

import json
from typing import Union, List, Tuple, Dict
from langchain.schema import AgentFinish

call_number = 0

def print_agent_output(agent_output: Union[str, List[Tuple[Dict, str]], AgentFinish], agent_name: str = 'Generic call'):
    global call_number  # Declare call_number as a global variable
    call_number += 1
    with open("crew_callback_logs.txt", "a") as log_file:
        if isinstance(agent_output, str):
            try:
                agent_output = json.loads(agent_output)
            except json.JSONDecodeError:
                pass

        if isinstance(agent_output, list) and all(isinstance(item, tuple) for item in agent_output):
            print(f"-{call_number}----Dict------------------------------------------", file=log_file)
            for action, description in agent_output:
                # Print attributes based on assumed structure
                print(f"Agent Name: {agent_name}", file=log_file)
                print(f"Tool used: {getattr(action, 'tool', 'Unknown')}", file=log_file)
                print(f"Tool input: {getattr(action, 'tool_input', 'Unknown')}", file=log_file)
                print(f"Action log: {getattr(action, 'log', 'Unknown')}", file=log_file)
                print(f"Description: {description}", file=log_file)
                print("--------------------------------------------------", file=log_file)

        # Check if the output is a dictionary as in the second case
        elif isinstance(agent_output, AgentFinish):
            print(f"-{call_number}----AgentFinish---------------------------------------", file=log_file)
            print(f"Agent Name: {agent_name}", file=log_file)
            agent_finishes.append(agent_output)
            # Extracting 'output' and 'log' from the nested 'return_values' if they exist
            output = agent_output.return_values
            # log = agent_output.get('log', 'No log available')
            print(f"AgentFinish Output: {output['output']}", file=log_file)
            # print(f"Log: {log}", file=log_file)
            # print(f"AgentFinish: {agent_output}", file=log_file)
            print("--------------------------------------------------", file=log_file)

        # Handle unexpected formats
        else:
            # If the format is unknown, print out the input directly
            print(f"-{call_number}-Unknown format of agent_output:", file=log_file)
            print(type(agent_output), file=log_file)
            print(agent_output, file=log_file)

In [ ]:
class material_generator():
    def entity_recognizer(self):
        return Agent(
                role = 'Named Entity Recogniser',
                goal = ''' Analyze the textual data and identify and classify
                        important information within text data.''',
                backstory='''You are a master at understanding the lectures''',
                llm = Groq_llm,
                verbose=False,
                allow_delegation=False,
                max_iter=5,
                memory=True,
                step_callback=lambda x: print_agent_output(x,'Named entity recognition'))

    def notes_maker(self):
        return Agent(
        role = 'Topic researcher agent',
        goal='''Analyze the topics given and make a complete notes accordingly.
                focus on key ideas don't try to write everything down word-for-word.
                Instead, focus on capturing the main points, definitions, arguments, and conclusions.
                Be selective: Not all details are essential. Use bullet points, abbreviations, and symbols
                to condense information efficiently.Use headings, subheadings, numbering, and indentation to create a clear hierarchy of information. ''',
        backstory = '''You are a master in math, physics, chemistry, biology, engineering subjects,
                    computer sciences and an expert in preparing study materials for students''',
        llm = Groq_llm,
        verbose=False,
        allow_delegation=False,
        memory=True,
        tools=[search_tool],
        step_callback=lambda x: print_agent_output(x,"notes_maker"),)

In [ ]:
class task_manager():
    # Define your tasks with descriptions and expected outputs
    def topic_reco(self, data):
        return Task(
            description="""Conduct a comprehensive analysis and grasp the general topic and identify your learning goals""",
            expected_output= 'a list of topics and learning goals',
            output_file=f"topics.txt",
            agent=ner_agent
            )

    def material_maker(self, data):
        return Task(
            description='''Analyze the topics given and make a complete notes accordingly.
                focus on key ideas don't try to write everything down word-for-word.
                Instead, focus on captu+ring the main points, definitions, arguments, and conclusions.
                Be selective: Not all details are essential. Use bullet points, abbreviations, and symbols
                to condense information efficiently.Use headings, subheadings, numbering, and indentation to create a clear hierarchy of information.Pleasen do not repeat sentences''',
            expected_output="""A large chuck of prepared notes""",
            context = [topic_model],
            output_file=f"output_list.txt",
            agent=data_gen
            )

In [ ]:

def generator(data):
      agents = material_generator()
      tasks = task_manager()
  #agents
      ner_agent = agents.entity_recognizer()
      data_gen = agents.notes_maker()

  #tasks
      topic_model = tasks.topic_reco(data)
      notes = tasks.material_maker(data)
      crew = Crew(
        agents=[ner_agent,data_gen],
        tasks=[topic_model, notes],
        verbose = 2,
        process = Process.sequential,
        full_output=True,
        share_crew = False,
        step_callback = lambda x: print_agent_output(x,"MasterCrew Agent"))
      results = crew.kickoff()
      d1 = list(results.values())
      tasks = d1[1]
      full_dictionary = []
      for i in tasks:
          newdict = dict(i)
          full_dictionary.append(newdict)
      return full_dictionary

In [ ]:
pip install reportLab

In [ ]:
notes = generator(data)

In [ ]:
https://153d-34-91-237-152.ngrok-free.app

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(public_url)

In [ ]:
pip install pyngrok

In [ ]:
!pip install pyngrok
from pyngrok import ngrok

!ngrok config add-authtoken 2fh3RRrTOnb4LIpFOyDh31DajTd_7T3TkvLzcjBaa9VBGhXw4

public_url = ngrok.connect(5000)
print("Print URL:", public_url)

In [ ]:
pip install face_recognition

In [ ]:
import face_recognition as fr
import cv2
import os
import pickle
import numpy as np

In [ ]:
import cv2 as cv
import os

class VideoPreprocessor:
    def __init__(self, directory, video_path):
        self.directory = directory
        self.video_path = video_path
        self.fps = None
        self.frame_count = 0
        self.extracted_frame_count = 0

    def empty_folder(self):
        for file in os.listdir(self.directory):
            file_path = os.path.join(self.directory, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        return True

    def extract_frames_per_second(self):
        try:
            cap = cv.VideoCapture(self.video_path)
            if not cap.isOpened():
                raise IOError("Error opening video!")

            self.fps = cap.get(cv.CAP_PROP_FPS)
            if self.fps <= 0:
                raise ValueError("Invalid FPS value.")

            if self.empty_folder():
                while True:
                    ret, frame = cap.read()

                    if not ret:
                        print("End of video reached.")
                        break

                    # Extract one frame per second
                    if self.frame_count % int(self.fps) == 0:
                        filename = f"{self.directory}/frame_{self.extracted_frame_count}.jpg"
                        cv.imwrite(filename, frame)
                        self.extracted_frame_count += 1

                    self.frame_count += 1

                print(f"Extracted {self.extracted_frame_count} frames to {self.directory}")

        except IOError as e:
            print(f"IOError: {e}")
        except ValueError as e:
            print(f"ValueError: {e}")
        finally:
            cap.release()



CLASSES TEST


In [ ]:
import face_recognition as fr
import cv2
import os
import pickle
import numpy as np

class FaceRecognition:
    def __init__(self, frames_dir, known_face_encodings, student_dic):
        self.frames_dir = frames_dir
        self.known_face_encodings = known_face_encodings
        self.students = student_dic

        csm_students = list(self.students.keys())

    def load_image_paths(self):
        image_paths = [os.path.join(self.frames_dir, filename) for filename in os.listdir(self.frames_dir) if filename.lower().endswith('.jpg')]
        return image_paths

    def find_faces(self, image_paths):
        face_locations = []
        face_encodings = []

        for path in image_paths:
            frame = fr.load_image_file(path)
            rgb_frame = frame[:, :, ::-1]  # Convert BGR to RGB

            face_locations = fr.face_locations(rgb_frame, number_of_times_to_upsample=1, model='hog')
            face_encodings_for_frame = fr.face_encodings(frame, face_locations)

            face_encodings.extend(face_encodings_for_frame)

        return face_encodings



    def cosine_similarity(self, vector_a, vector_b):
        vector_a = np.array(vector_a, dtype=float)
        vector_b = np.array(vector_b, dtype=float)
        dot_product = np.dot(vector_a, vector_b)
        magnitude_a = np.linalg.norm(vector_a)
        magnitude_b = np.linalg.norm(vector_b)

        if magnitude_a == 0 or magnitude_b == 0:
            return 0

        cosine_similarity_value = dot_product / (magnitude_a * magnitude_b)
        return cosine_similarity_value

    def compare_faces(self, video_face_encodings):
        similarities = []
        student_names = []
        csm_students = list(self.students.keys())

        for y in video_face_encodings:
            nec = [self.cosine_similarity(y, x) for x in self.known_face_encodings]
            similarities.append(nec)

        for x in similarities:
            student_names.append(csm_students[np.argmax(x)])

        return list(set(student_names))

In [ ]:
from Face_recog_m

In [ ]:
from flask import *
from VideoPreoprocess import VideoPreprocessor
from Face_recog_methods_2 import FaceRecognition
import sqlite3
import numpy as np
import os
app = Flask(__name__)
app.secret_key="I_SAVE_GOTHAM"
@app.route('/')
def videoUploadFile():
    return render_template('index.html')

ALLOWED_EXTENSIONS = ['mp4']

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route("/upload",methods = ['POST',"GET"])
def uploadStudents():
    if 'video' not in request.files:
        return  jsonify("No Video Found")
    video = request.files['video']
    if video.filename == "":
        return jsonify("No File Selected")
    if video and allowed_file(video.filename):
        video.save(video.filename)
        video_object = VideoPreprocessor("Face-Attendance-Server-main/Frames",video_path=video.filename)
        if video_object.empty_folder():
            video_object.extract_frames_per_second()
            studentdetails,known_encodings = fetchFromDataBase()
            face_object = FaceRecognition("Face-Attendance-Server-main/Frames",known_encodings,studentdetails)
            new_image_paths = face_object.load_image_paths()
            new_face_encodings = face_object.find_faces(new_image_paths)
            attendees_list = face_object.compare_faces(new_face_encodings)
            print(attendees_list)
            return jsonify(attendees_list)
        return jsonify("Error in emptying the Folder")
    return jsonify("Invalid type in the File ")
def fetchFromDataBase():
    with sqlite3.connect("Attendance.db") as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM USERS")
        student_dict = {}
        new_Face_Encodings = []
        for i in cursor :
            student_dict[i[1]] = i[0]
            new_Face_Encodings.append([float(j) for j in i[2].split("/") ])
    return student_dict,new_Face_Encodings

if __name__ == "__main__":
    app.run(debug=True)
